In [ ]:
!pip install tinyec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tinyec: filename=tinyec-0.4.0-py3-none-any.whl size=20877 sha256=e4ba055af420751acb1c12c238ca26295c242741b7fc97729bb9e243ef598f13
  Stored in directory: /root/.cache/pip/wheels/02/37/a5/aa011cfa66451de6aa2dbccaa3e7862e8290f0946653753265
Successfully built tinyec


In [ ]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pickle
# Descarga de librerías para la generación de llaves
from tinyec import registry
import secrets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/Shareddrives/Cripto 3/reto/Prosumer_ABC.csv', header= 0, sep = ";" )
dataset.head(3)

,ID,Consumo (0) / Producción (1),Dia,Mes,Año,1,2,3,4,5,...,87,88,89,90,91,92,93,94,95,96
0,ABC,0,2,11,2013,58.00,75.0,65.0,0.08,67.0,...,338.0,369.0,318.0,322.0,345.0,298.0,316.0,299.0,341.00,304.0
1,ABC,0,3,11,2013,0.16,0.1,68.0,69.00,75.0,...,67.0,74.0,61.0,84.0,74.0,67.0,77.0,91.0,0.07,61.0
2,ABC,0,4,11,2013,0.08,66.0,77.0,64.00,84.0,...,114.0,106.0,128.0,511.0,74.0,84.0,59.0,69.0,69.00,71.0


In [ ]:
import numpy as np
import hashlib

In [ ]:
# algritmo euclideano extendido 
# a - grupo
# b - elemento
def alg_euc_ext(a,b):
  if b == 0:
    d = a; x = 1; y =0
    return d,x, y
  
  x1 = 0; x2 = 1; y1 = 1; y2 = 0

  while b != 0:
    q = int(a/b)
    r = a - q*b
    x = x2 - q*x1
    y = y2 - q*y1
    a = b
    b = r
    x2 =x1
    x1 = x
    y2 = y1
    y1 = y

  d = a; x = x2; y = y2
  # y es inverso de a
  return d,x,y

def firmado_dsa(curve, m):
  # Generación de clave
  q = curve.field.n
  P = curve.g # element G of prime order q is chosen in E(GF(p))
  x = secrets.randbelow(q) #privkey
  Q = x * P # pubkey
  #h = hashlib.sha1(m)
  h = int(hashlib.sha512(m).hexdigest(),16)
  #Firma digital
  r = 0
  s = 0

  while r == 0 or s == 0:
    k = secrets.randbelow(q)
    kP = k * P
    r = kP.x % q 
    l = alg_euc_ext(q, k)[2] # inverso de k
    s = (l*(h + x*r))%q 
  return r, s, Q, h

def verificado_dsa(curve, r, s, Q, h):
  #Generaicón de clave
  q = curve.field.n
  P = curve.g
  
  #Verificación de la firma
  if 1<=r and 1<=s and r<=(q-1) and s<=(q-1):
    w = alg_euc_ext(curve.field.n, s)[2] # invserso s
    u1 = h * w % q
    u2 = r * w % q
    v = (u1*P + u2*Q).x%q
    if v == r:
      print('Se acepta la firma')
    else:
      print('Se rechaza la firma')

  else:
    print('Error: no cumple con 1<=r, s<=q-1')

## Ejemplo con curva

In [ ]:
curve = registry.get_curve('brainpoolP256r1')
# Se convierte la base de datos a bytes para su manejo
m = pickle.dumps(dataset, protocol=4)

r,s,Q,h= firmado_dsa(curve, m)

verificado_dsa(curve, r, s, Q, h)

Se acepta la firma


## Por renglón

In [ ]:
# Firmado y lista con la curva, r, s y Q
def firmado_df(df):
  a = []
  for i in df.index:
    curve = registry.get_curve('brainpoolP256r1')
    m = pickle.dumps(df.iloc[i].to_numpy(), protocol=4)
    r,s,Q,h = firmado_dsa(curve, m)
    a.append([curve, r,s,Q,h])
  return a

In [ ]:
# Verificación firma
def ver_df(a):
  for i in range(len(a)):
    verificado_dsa(a[i][0],a[i][1],a[i][2],a[i][3],a[i][4])

In [ ]:
ver_df(firmado_df(dataset.iloc[0:5]))

Se acepta la firma
Se acepta la firma
Se acepta la firma
Se acepta la firma
Se acepta la firma
